In [1]:
import pandas as pd
import numpy as np
import n2n4m.preprocessing as preprocessing

In [76]:
df = pd.read_json("../data/full_pixel_dataset.json")

In [ ]:
df = preprocessing.expand_dataset(df)
df = preprocessing.drop_bad_bands(df)